In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics_class import Metrics
import shutil
from open3d.pipelines import registration
from predict_maskpls import RefinerModel

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [ ]:

'''
if os.path.exists('{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)
'''

This cell can be ignored after first run as outputs are stored 

In [ ]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed.npz',panoptic_nonground=kitti_labels['panoptic_nonground'],
                                                panoptic_ground=kitti_labels['panoptic_ground'],
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )


In [ ]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}.pcd')

kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}.npz') as data :
        kitti_labels_orig['panoptic_ground'] = data['panoptic_ground']
        kitti_labels_orig['panoptic_nonground'] = data['panoptic_nonground']
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_{0}.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

In [ ]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

Now we subsample the poses based on a voxel_size

In [ ]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

In [ ]:
import sklearn
from sklearn.cluster import DBSCAN

def DBSCAN_clustering_logic(cur_pcd, pcd_all, eps=0.3, min_samples=10):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    not_road_points = np.asarray(cur_pcd.points)
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(not_road_points)
    #clustering = DBSCAN(min_cluster_size=100).fit(not_road_points)
    labels_not_road = clustering.labels_
    colors_gen = generate_random_colors(500)
    
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(pcd_all.points), 1)) * -1
    labels_non_ground = kDTree_1NN_feature_reprojection(cluster_labels, pcd_all, labels_not_road.reshape(-1,1), cur_pcd)
    colors = np.zeros((labels_non_ground.shape[0],3))
    unique_labels = list(np.unique(labels_non_ground))
    for j in unique_labels:
            cur_idcs = np.where(labels_non_ground == j)[0]
            
            colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
    pcd_all.colors = o3d.utility.Vector3dVector(colors / 255.)
    return pcd_all

def dbscan_clustering(pcd_chunks_major_downsampled, pcds, center_ids,ground_clouds):
    labels_clustering = []
    for i in range(len(pcd_chunks_major_downsampled)):
        cur_pcd = pcd_chunks_major_downsampled[i]
        pcd_all = pcds[i]
        ground_cloud = ground_clouds[i]
        cluster_labels = DBSCAN_clustering_logic(cur_pcd, pcd_all)  # Implement your DBSCAN logic here
        ground_labels = np.ones((np.asarray(ground_cloud.points).shape[0],1)) * -1
        cluster_labels = np.concatenate((cluster_labels,ground_labels),0)
        labels_clustering.append(cluster_labels)
        
    return labels_clustering


In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [ ]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0



out_dbscan = 'out_dbscan/'
if os.path.exists(out_dbscan) == True : 
        shutil.rmtree(out_dbscan)
        
os.makedirs(out_dbscan)

out_kitti = 'out_kitti/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = 'out_kitti_instance2/'
#if os.path.exists(out_kitti_instance) == True : 
#        shutil.rmtree(out_kitti_instance)
#os.makedirs(out_kitti_instance)

        
out_refined = 'out_refined/'
if os.path.exists(out_refined) == True : 
        shutil.rmtree(out_refined)
os.makedirs(out_refined)

limit = 2 ##use this for experiments to run limit chunks numberss

maskpls = RefinerModel()
eval_refiner = True 

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                name = str(center_ids[sequence]).zfill(6) + '.pcd'
                if eval_refiner == False : 
                        merged_chunk,file_name, pcd_chunk, pcd_chunk_ground, inliers_ground = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                                pcd_nonground_chunks_major_downsampling,
                                pcd_nonground_minor,T_pcd,center_positions,center_ids,
                                positions,first_position,sampled_indices_global,
                                chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                                alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                                proximity_threshold=proximity_threshold,
                                out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                                patchwise_indices=patchwise_indices)
                        o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                        kitti_labels['ground']['panoptic'][sequence] = kitti_labels['ground']['panoptic'][sequence][inliers_ground]
                        kitti_labels['ground']['instance'][sequence] = kitti_labels['ground']['instance'][sequence][inliers_ground]
                        #kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['panoptic'][sequence].reshape(-1,),
                        #                        colors=colors,gt_labels=kitti_labels_orig['panoptic_nonground'])
                        #kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                        #                        colors=colors,gt_labels=kitti_labels_orig['instance_nonground'])
                        #o3d.io.write_point_cloud(out_kitti + name, kitti_chunk + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                        #o3d.io.write_point_cloud(out_kitti_instance + name, kitti_chunk_instance + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                else :  
                        print("sequence",sequence)
                        cur_pcd = o3d.io.read_point_cloud(out_kitti_instance + name)
                        predicted_refined = maskpls.forward_and_project(cur_pcd)
                        #maskpls.project_pseudo_labels(cur_pcd,center_ids[sequence],name)
                        o3d.io.write_point_cloud(out_refined + name, predicted_refined,write_ascii=False, compressed=False, print_progress=False)
                
                        #pcd_dbscan = DBSCAN_clustering_logic(cur_pcd,
                        #                        eps=0.3, min_samples=10)
                        #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                
                
                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                

        
        
        

In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds


def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd


def merge_chunks_unite_instances_conf(chunks: list, confs_dict=None,icp=False):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:

        if icp:
            last_chunk.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
            new_chunk.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
            reg_p2l = registration.registration_icp(new_chunk, last_chunk, 0.9, np.eye(4), registration.TransformationEstimationPointToPlane(), registration.ICPConvergenceCriteria(max_iteration=1000))
            transform = reg_p2l.transformation
            new_chunk.transform(transform)

        points_1 = np.asarray(last_chunk.points)
        points_2 = np.asarray(new_chunk.points)

        colors_1 = np.asarray(last_chunk.colors)
        colors_2 = np.asarray(new_chunk.colors)

        unique_colors_1 = np.unique(colors_1, axis=0)
        unique_colors_2 = np.unique(colors_2, axis=0)

        instance2point_1 = {}
        for i in range(unique_colors_1.shape[0]):
            if not np.all(unique_colors_1[i] == 0.0): # Streets are black
                instance2point_1[i] = {}
                inds = np.where(np.all(colors_1 == unique_colors_1[i], axis=1))[0]
                instance2point_1[i]["points"] = points_1[inds]
                instance2point_1[i]["inds"] = inds

        instance2point_2 = {}
        for i in range(unique_colors_2.shape[0]):
            if not np.all(unique_colors_2[i] == 0.0): # Streets are black
                instance2point_2[i] = {}
                inds = np.where(np.all(colors_2 == unique_colors_2[i], axis=1))[0]
                instance2point_2[i]["points"] = points_2[inds]
                instance2point_2[i]["inds"] = inds
        
        id_pairs_iou = []
        for id_1, entries_1 in instance2point_1.items():
            points1 = entries_1["points"]
            min_bound = np.min(points1, axis=0)
            max_bound = np.max(points1, axis=0)
            association = []
            for id_2, entries_2 in instance2point_2.items():
                points2 = entries_2["points"]
                intersection = np.where(np.all(points2 >= min_bound, axis=1) & np.all(points2 <= max_bound, axis=1))[0].shape[0]
                if intersection > 0:
                    union = len(np.unique(np.concatenate((points1, points2))))
                    iou = float(intersection) / float(union)
                    if iou > 0.01:
                        association.append((id_2, iou))
            if len(association) != 0:
                for (association_id, iou) in association:
                    id_pairs_iou.append((id_1, (association_id, iou)))
        
        ids_chunk_1 = []
        ids_chunk_2 = []
        ious = []
        for id1, (id2, iou) in id_pairs_iou:
            if id2 not in ids_chunk_2:
                ids_chunk_1.append(id1)
                ids_chunk_2.append(id2)
                ious.append(iou)
            else:
                i = ids_chunk_2.index(id2)
                if iou > ious[i]:
                    ious[i] = iou
                    ids_chunk_1[i] = id1

        for id1, id2 in zip(ids_chunk_1, ids_chunk_2):
            inds2 = instance2point_2[id2]["inds"]
            colors_2[inds2] = unique_colors_1[id1]

        new_chunk_recolored = o3d.geometry.PointCloud()
        new_chunk_recolored.points = new_chunk.points
        new_chunk_recolored.colors = o3d.utility.Vector3dVector(colors_2)
        last_chunk = new_chunk_recolored

        merge += new_chunk_recolored
        merge.remove_duplicated_points()

    return merge

In [ ]:
out_dbscan = 'out_dbscan/'
out_kitti = 'out_kitti/'
out_kitti_instance = 'out_kitti_instance/'


point_clouds = get_merge_pcds(out_folder_ncuts)
point_clouds_dbscan = get_merge_pcds(out_dbscan)[:-1]
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
point_clouds_refined = get_merge_pcds(out_refined)[:-1]

#merge = merge_chunks_unite_instances(point_clouds)
#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
#merge_kitti = merge_unite_gt(point_clouds_kitti)
#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
merge_refined = merge_chunks_unite_instances(point_clouds_refined)

#o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_refined.pcd", merge_refined, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)

#unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)



In [ ]:
'''
optional cell for loading stored files
import open3d as o3d

out_folder = 'pcd_preprocessed/'
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')
merge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')
merge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
merge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')

print(merge)
print(merge_kitti)
print(merge_exp2)
'''

In [ ]:
#new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'kitti_labels.pcd')
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined.pcd')
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

new_refined = remove_semantics_pcd(merge_refined,labels_kitti,labels_refined)

In [ ]:
#o3d.visualization.draw_geometries([new_refined])


In [ ]:
unique_colors, labels_refined = np.unique(np.asarray(new_refined.colors),axis=0, return_inverse=True)
label_to_confidence = {}
pcd_cols = np.asarray(new_refined.colors) 
for label in list(np.unique(labels_refined)):
	idcs = np.where(labels_refined == label)[0]
	cur_color = pcd_cols[idcs[0]]
	key = str(int(cur_color[0] * 255)) + '|' + str(int(cur_color[1] * 255)) + '|' + str(int(cur_color[2] * 255))
	label_to_confidence[label] = maskpls.confs_dict[key]

In [ ]:
metrics_refined = Metrics(name='refined')


metrics_refined.update_stats(labels_refined,labels_kitti,label_to_confidence,calc_all=False)


In [ ]:
o3d.visualization.draw_geometries([merge_refined])

In [ ]:
out_folder = 'pcd_preprocessed/'
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
#unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
#unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

#new_refined = remove_semantics(labels_kitti,labels_refined)
#o3d.visualization.draw_geometries([color_pcd_by_labels(merge_refined,new_refined)])

In [ ]:
o3d.visualization.draw_geometries([merge_refined])

In [ ]:
fn = out_folder + "merge_refined.pcd"
fn2 = out_folder + 'ncuts_tarl_spatial1_all.pcd'
pcd = o3d.io.read_point_cloud(fn)
pcd_ncuts = o3d.io.read_point_cloud(fn2)
o3d.visualization.draw_geometries([pcd])